In [3]:
import os
os.getcwd()

'C:\\Users\\tkeo1\\OneDrive\\바탕 화면\\coding\\Python'

In [4]:
os.chdir("C:/Users/tkeo1/OneDrive/바탕 화면/coding/Python/data/")

In [17]:
import tensorflow as tf
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',
    image_size=(150,150),
    batch_size=64,               #이미지 2만장 한번에 넣지 않고 batch 숫자만큼 넣음
    subset='training',
    validation_split=0.2,
    seed=1234
) # 이미지 숫자화 함수 , 폴더 내 이미지들을 바로 dataset 만들어줌

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset',
    image_size=(150,150),
    batch_size=64,               #이미지 2만장 한번에 넣지 않고 batch 숫자만큼 넣음
    subset='validation',
    validation_split=0.2,
    seed=1234
) # 이미지 숫자화 함수 , 폴더 내 이미지들을 바로 dataset 만들어줌

print(train_ds)

def 전처리함수(i,정답):
    i = tf.cast( i/255.0, tf.float32)
    return i, 정답

train_ds=train_ds.map(전처리함수)
val_ds=val_ds.map(전처리함수)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [18]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
inception_model=InceptionV3(input_shape=(150,150,3), include_top=False,weights=None) #include_top 마지막 dense layer
inception_model.load_weights('C:/Users/tkeo1/Downloads/inception_v3.h5')

#inception_model.summary()

for i in inception_model.layers: # w값 고정하는 방법 학습금지레이어 설정
    i.trainable=False

unfreeze=False
for i in inception_model.layers:
    if i.name=='mixed6':
        unfreeze=True
    if i.name==True:
        i.trainable=True

마지막레이어=inception_model.get_layer('mixed7')

print(마지막레이어)
print(마지막레이어.output)
print(마지막레이어.output_shape)

KerasTensor(type_spec=TensorSpec(shape=(None, 7, 7, 768), dtype=tf.float32, name=None), name='mixed7/concat:0', description="created by layer 'mixed7'")
(None, 7, 7, 768)


In [19]:
import tensorflow as tf

layer1=tf.keras.layers.Flatten()(마지막레이어.output)
layer2=tf.keras.layers.Dense(1024,activation='relu')(layer1)
drop1=tf.keras.layers.Dropout(0.2)(layer2)
layer3=tf.keras.layers.Dense(1,activation='sigmoid')(drop1)

model=tf.keras.Model(inception_model.input,layer3)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(train_ds,validation_data=val_ds,epochs=2)

Epoch 1/2
313/313 [==============================] - 146s 457ms/step - loss: 0.1710 - acc: 0.9531 - val_loss: 0.0806 - val_acc: 0.9660
Epoch 2/2
313/313 [==============================] - 135s 430ms/step - loss: 0.0429 - acc: 0.9842 - val_loss: 0.0743 - val_acc: 0.9714


In [20]:
model.compile(loss='binary_crossentropy',optimizer=tf.keras.optimizers.Adam(lr=0.00001),metrics=['acc'])
model.fit(train_ds,validation_data=val_ds,epochs=2)

Epoch 1/2


C:\Users\tkeo1\anaconda3\envs\tf\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


313/313 [==============================] - 138s 429ms/step - loss: 0.0148 - acc: 0.9956 - val_loss: 0.0774 - val_acc: 0.9720
Epoch 2/2
313/313 [==============================] - 134s 428ms/step - loss: 0.0120 - acc: 0.9965 - val_loss: 0.0796 - val_acc: 0.9730
